In [2]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import re
import random
import pandas as pd

In [3]:
data = pd.read_csv('Textranked.csv')
data.head()

# data.dropna(inplace=True)
# data.rename(columns={'본문':'question', '답변':'answer'}, inplace=True)
# data.to_csv('prep_data_dotsplits.csv', index=False, encoding='utf-8')

,Question,Answer
0,정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다. 아버지가 제 명의로 사업자...,차량의 경우 현재 점유자인 아버지를 피고로 하여 차량 인도 청구소송 및 차량 점유 ...
1,년 전에 대포 통장이로 인해 피해를 봐서 재판 결과가 불기소 유통 지을 받았거든요....,소장 내용에 대한 검토가 필요하므로 소장을 지참하고 상담을 받으시기 바랍니다
2,제 계좌로 입금을 했다던가 그런부분 전혀없이 그 형의 계좌로 그 형의 공인인증서를 ...,별도의 구체적 상담이 필요하나 실제 채무가 없음에도 공증을 가지고 채권자가 강제집행...
3,채권시효년 소멸로 본다면 어떤 대응이 가능할지 여쭤봅니다 지인에게 연대보증 해줬는데...,별도의 구체적 상담이 필요하나 이행 권고 결정은 기판력의 제한이 없으므로 소멸시효 ...
4,그 와중에 피의자와 어떻게 연락이 닿게 되어서 갚겠다는 의사를 보였고 정말 적은 ...,가해자가 위 사건으로 구속 기소되었다면 가해자를 피고로 하여 민사소송 및 강제집행으...


In [4]:
SOS_token = 0
EOS_token = 1

class QNA:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1:"EOS"}
        self.n_words = 2

    def addSentence(self, sentence):
        for word in sentence.split('.'):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
def normalizeString(s):
    s = re.sub(r"([.!?])", r" ", s)
    s = re.sub(r"[^가-힣.!?]+", r" ", s)
    return s

In [6]:
def readchat(question, answer, reverse=False):
    print("Reading lines...")

    lines = open('prep_data_dotsplits.csv', encoding='utf-8').read().strip().split('\n')[1:]
    
    pairs = [[normalizeString(s) for s in l.split(',')] for l in lines]

    if reverse:
        paris = [list(reversed(p)) for p in pairs]
        input_q = QNA(answer)
        output_a = QNA(question)
    else:
        input_q = QNA(question)
        output_a = QNA(answer)
    
    return input_q, output_a, pairs

In [7]:
MAX_LENGTH = 200
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [8]:
def prepareData(question, answer, reverse=False):
    input_q, output_a, pairs = readchat(question, answer, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    for pair in pairs:
        input_q.addSentence(pair[0])
        output_a.addSentence(pair[1])
    print("Counted words:")
    print(input_q.name, input_q.n_words)
    print(output_a.name, output_a.n_words)
    return input_q, output_a, pairs

input_q, output_a, pairs = prepareData('question', 'answer', True)
print(random.choice(pairs))

Reading lines...
Read 18165 sentence pairs
Counted words:
answer 14181
question 14019
['다음 주 정도에 소장 제출을 하려고 하는데요 청구원인은 그런대로 써둔 상태입니다 밑에 제가 작성한 청구취지를 검토하여 주실 수 있으실까요 피고는 원고에게 원 및 이에 대하여 판결정본 도달 다음 날부터 다 갚는 날까지 연 의 비율로 계산한 돈을 지급하라 소송의 비용은 피고의 부담으로 한다 위 항은 가집행할 수 있다 보통은 소장 부본 송달 다음 날부터 틀 이자 기준일로 잡지만 저는 위로금 지급 소송이라서 소장 부본 송달일이 아닌 판결정본 도들 일을 기준으로 잡았는데요 보정권고 명령이 내려올까요 소송이 진행되는 동안 최소 개월은 판결문 발급일까지 걸릴 것 같아서 그동안 이자도 부담스러울 수도 있기 때문에 판결정본 도들 일을 이자 발생 기준일로 잡았는데요 혹시 검수해주실 수 있나요', '민사소송 손해배상 청구취지 기재해를 질문하셨습니다 질문 원문 피고는 원고에게 원 및 이에 대하여 판결정본 도달 다음 날부터 다 갚는 날까지 연 의 비율로 계산한 돈을 지급하라 소송의 비용은 피고의 부담으로 한다 위 항은 가집행할 수 있다 권고 수정본 피고는 원고에게 금 원 및 이에 대하여부터 이 사건 소장 부본 송달일 까지는 연 의 그다음 날부터 다 나포는 날까지는 연 의 비율로 각 계산한 돈을 지급하라 소송 비용은 피고의 부담으로 한다 위 제 항은 가집행할 수 있다 에는 돈을 받으시기로 정한 변제기를 적으시면 됩니다 손해배상 소송이라면 손해의 발생 일을 적으시면 됩니다 두 분 중 한 분이 상인이라면 상법상 이율인 로 기재해도 됩니다 소장 부본 송달 다음날부터 다 갚는 날까지는 서촉 법상 이율이 적용됩니다 온라인 상담의 한계상 위 답변은 구체적 상담에 기초한 답변이 아니므로 참조만 하시기 바랍니다']


In [9]:
pairs

[['정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다 부디 자세한 설명해주시면 감사하겠습니다 아버지가 제 명의로 사업자를 내서 사업을 했으나 늘어나는 것은 빛뿐이었습니다 더 이상은 방관할 수가 없어 아버지에게 제 명의로 해드린 것을 모두 정지시키려고 하는 상황입니다 또 어머니 명의의 집을 팔아서 빚을 메꾸려고 합니다 첫 번째 질문입니다 어머니 명의로 된 집 아버지 어머니 함께 거주를 매도하려고 하는데 그 과정에서 아버지가 방해를 하고 가구 이동을 못 시키게 하고 그러면 어떤 방법을 취할 수 있을까요 두 번째 질문입니다 제 명의로 된 차량을 아버지가 사용하고 계셨는데 제가 도로 갖고 와도 될까요 아니면 어떤 방법이 있을까요 답변해 주시면 정말 감사드리겠습니다',
  '차량의 경우 현재 점유자인 아버지를 피고로 하여 차량 인도 청구소송 및 차량 점유 이전금지 가처분으로 차량을 조기에 회수할 수 있습니다'],
 [' 년 전에 대포 통장이로 인해 피해를 봐서 재판 결과가 불기소 유통 지을 받았거든요 근데 년 뒤 오늘 법원에서 송달이 왔어요 상대편에서 부당이득금 기소를 했더라고요 년 전에 불기소 됐으니 걱정하시지 말라고 불기소 이유통지받았던 서류도 있거든요 좀 전에 대한 법률공단에도 상담 신청을 해놓은 상태인데 어떻게 해야 할까요',
  '소장 내용에 대한 검토가 필요하므로 소장을 지참하고 상담을 받으시기 바랍니다'],
 [' 이행권고결정판결 채권압류 및 추심명령 인용 채권추심시중은행 통장 압류 및 추심됨 결정문송달받음 채권시효 년 소멸로 본다면 어떤 대응이 가능할지 여쭤봅니다 지인에게 연대보증 해줬는데 잠적해서 채권자가 전자소송으로 승소하였습니다 저도 대처 못함 번 소송결과물로 번을 집행한거 같은데요 저때 이후로 법원에서 서류하나 받은적은 없습니다 대법원인터넷등기소 열람해봐도 잘 찾기도 힘들지만 소송건 보이는건 없구요 전자소송 홈페이지에서도 열람되는게 없습니다 마지막 기록일자로 계산해봐도 지금 시점은 년이 넘어 년이 되어가려고 하는데요 열심히 네이버 찾아봐도 

In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

In [11]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

In [12]:
def indexesFromSentence(QNA, sentence):
    return [QNA.word2index[word] for word in sentence.split('.')]


def tensorFromSentence(QNA, sentence):
    indexes = indexesFromSentence(QNA, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=DEVICE).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_q, pair[0])
    target_tensor = tensorFromSentence(output_a, pair[1])
    return (input_tensor, target_tensor)

In [13]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=DEVICE)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=DEVICE)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing 포함: 목표를 다음 입력으로 전달
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Teacher forcing 미포함: 자신의 예측을 다음 입력으로 사용
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # 입력으로 사용할 부분을 히스토리에서 분리

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [14]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [15]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [16]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [17]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=DEVICE)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=DEVICE)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [18]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [19]:
hidden_size = 256
encoder1 = EncoderRNN(input_q.n_words, hidden_size).to(DEVICE)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_a.n_words, dropout_p=0.1).to(DEVICE)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

4m 39s (- 65m 9s) (5000 6%) 4.8282
9m 52s (- 64m 8s) (10000 13%) 4.7709
15m 15s (- 61m 3s) (15000 20%) 4.7584
20m 48s (- 57m 13s) (20000 26%) 4.7496


In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

NameError: name 'evaluateRandomly' is not defined

In [72]:
pairs

[['정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다 부디 자세한 설명해주시면 감사하겠습니다 아버지가 제 명의로 사업자를 내서 사업을 했으나 늘어나는 것은 빛뿐이었습니다 더 이상은 방관할 수가 없어 아버지에게 제 명의로 해드린 것을 모두 정지시키려고 하는 상황입니다 또 어머니 명의의 집을 팔아서 빚을 메꾸려고 합니다 첫 번째 질문입니다 어머니 명의로 된 집 아버지 어머니 함께 거주를 매도하려고 하는데 그 과정에서 아버지가 방해를 하고 가구 이동을 못 시키게 하고 그러면 어떤 방법을 취할 수 있을까요 두 번째 질문입니다 제 명의로 된 차량을 아버지가 사용하고 계셨는데 제가 도로 갖고 와도 될까요 아니면 어떤 방법이 있을까요 답변해 주시면 정말 감사드리겠습니다',
  '차량의 경우 현재 점유자인 아버지를 피고로 하여 차량 인도 청구소송 및 차량 점유 이전금지 가처분으로 차량을 조기에 회수할 수 있습니다'],
 [' 년 전에 대포 통장이로 인해 피해를 봐서 재판 결과가 불기소 유통 지을 받았거든요 근데 년 뒤 오늘 법원에서 송달이 왔어요 상대편에서 부당이득금 기소를 했더라고요 년 전에 불기소 됐으니 걱정하시지 말라고 불기소 이유통지받았던 서류도 있거든요 좀 전에 대한 법률공단에도 상담 신청을 해놓은 상태인데 어떻게 해야 할까요',
  '소장 내용에 대한 검토가 필요하므로 소장을 지참하고 상담을 받으시기 바랍니다'],
 [' 이행권고결정판결 채권압류 및 추심명령 인용 채권추심시중은행 통장 압류 및 추심됨 결정문송달받음 채권시효 년 소멸로 본다면 어떤 대응이 가능할지 여쭤봅니다 지인에게 연대보증 해줬는데 잠적해서 채권자가 전자소송으로 승소하였습니다 저도 대처 못함 번 소송결과물로 번을 집행한거 같은데요 저때 이후로 법원에서 서류하나 받은적은 없습니다 대법원인터넷등기소 열람해봐도 잘 찾기도 힘들지만 소송건 보이는건 없구요 전자소송 홈페이지에서도 열람되는게 없습니다 마지막 기록일자로 계산해봐도 지금 시점은 년이 넘어 년이 되어가려고 하는데요 열심히 네이버 찾아봐도 